In [1]:
import numpy as np
import scipy.io
from midiutil import MIDIFile

In [5]:
key_time = 50e-3*0.5
Fs = 48000

keys = scipy.io.loadmat('MATLAB_data/31Jan_jc_upper_octave.mat')['key']
notes_list = scipy.io.loadmat('notes_88.mat')['notes']
freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

In [ ]:
class Note:
    def __init__(self, pitch, start, end, intensity, channel = -1):
        self.pitch = pitch
        self.start = start
        self.end = end
        self.intensity = intensity
        self.channel = channel

In [6]:
def get_intensity(cur_key,cur_freq,wave_freq):
    if cur_key == 1:
        sigma = notes_list[1]-notes_list[0]
    else:
        sigma = notes_list[cur_key-1]-notes_list[cur_key-2]
    sigma = sigma/2
    intensity = np.exp(-0.5 *((cur_freq-wave_freq)/sigma)**2)
    #print('cur', cur_freq,'wave',wave_freq)
    return intensity

In [ ]:
def picknotes(cur_key,cur_freq):
    j = 0
    while j < len(cur_key):
        if cur_key[j] != 0:
            pitch = cur_key[j]
            start = j
            while cur_key[j] != 0 and j < len(cur_key)-1:
                j += 1
            intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
            notes.append(Note(pitch, start, j - 1, intensity))
        j += 1

In [8]:
notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_freq = freq_list[i]
    picknotes(cur_key,cur_freq)

IndexError: index 1784 is out of bounds for axis 0 with size 1784

In [ ]:
print(len(notes))

In [3]:
notes = sorted(notes, key = lambda note: note.start)
notes = sorted(notes, key = lambda note: -note.pitch)

NameError: name 'notes' is not defined

In [4]:
def remove_repetitive(notes):
    note_set = [0]*88
    i = 0
    while i < len(notes):
        note = notes[i]
        if note_set[note.pitch-1] == 0:
            note_set[note.pitch-1] = [(note.start,note.end)]
        else:
            if (note.start,note.end) in note_set[note.pitch-1]:
                del notes[i]
                i -= 1
            else:
                note_set[note.pitch-1].append((note.start,note.end))
        i += 1
    return notes

In [ ]:
for note in notes:
    print(note)

In [ ]:
new_notes = remove_repetitive(notes)
for note in new_notes:
    print(note)

In [ ]:
def multichannel(notes):    
    channel = [0]*88
    start_time = [-1]*88
    for note in notes:
        if channel[note.pitch-1]==0:
            start_time[note.pitch-1]=note.start
            note.channel=channel[note.pitch-1]
            channel[note.pitch-1] += 1
        else:
            if note.start - start_time[note.pitch-1] < 10:
                note.channel=channel[note.pitch-1]
                channel[note.pitch-1] += 1
            else:
                channel[note.pitch-1] = 0
                start_time[note.pitch-1]=note.start
                note.channel=channel[note.pitch-1]
                channel[note.pitch-1] += 1
    return notes

In [ ]:
nnew_notes = multichannel(new_notes)
for note in nnew_notes:
    print(note)

In [ ]:
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 1200)

In [ ]:
for note in nnew_notes:
    mf.addNote(track,note.channel,note.pitch+20, note.start, note.end-note.start+1,100)

In [ ]:
with open("midifile/31Jan_jc_channel_upper_octave.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
print(len(nnew_notes))
print(len(notes))

In [7]:
def picknotes_old(cur_key,cur_freq):
    j = 0
    while j < len(cur_key):
        if cur_key[j] != 0:
            
            d = {}
            d['note'] = cur_key[j]
            d['start'] = j
            while cur_key[j] != 0 and j < len(cur_key)-1:
                j += 1
            d['end'] = j - 1
            intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
            d['intensity'] = intensity
            notes.append(d)
        j += 1